# Decorators 

You've probably already heard of **decorators**. This is a feature that is widely used in the Python world. They allow you to write concise, readable and non-repetitive code.

Concretely, a decorator is a callable that takes another callable as an argument and returns a copy of the same callable by "decorating" it, i.e. pre-processing and/or post-processing it. In the jargon, this is often referred to as wrapping (wrapping, sprinkling, adding something extra, decorating).

In [1]:
def my_decorator(my_function):
    print(my_function.__name__)
    return my_function

To apply a decorator, we precede the definition line of the function to be decorated with a line containing an `@` character, then the name of the decorator to be applied, for example :

In [2]:
@my_decorator
def addition(a, b):
    return a + b

addition


In [3]:
addition(5, 6)

11

It is therefore clear that the decorator is applied when the function is defined, and not when it is called. Here we use a very simple decorator that returns the same function that it got as argument (`my_function`), but it could very well be that it returns another one, which would for example be created on the fly.

This has the effect of replacing `addition` by the return of the decorator function called with `addition` as argument, which is strictly equivalent to the following.



In [4]:
def addition(a, b):
    return a + b


addition = my_decorator(addition)

addition


Let's say we would like to change our `addition` function to display the operands and then the result, without touching the body of our function. We can make a decorator that will return a new function that will display the arguments, call our original function, then display the return and return it (to keep the original behavior).

Thus, our decorator becomes :

In [5]:
def print_decorator(function):
    def new_function(a, b):  # New function behaving as the `function` to be decorated
        print(f"Addition of numbers {a} and {b}")
        return_value = function(a, b)  # Call to `function`
        print(f"Result: {return_value}")
        return return_value

    return new_function  # don't forget to return `new_function`

If we now apply this decorator to our addition function:

In [25]:
@print_decorator
def addition(a, b):
    return a + b


In [26]:
addition(1, 2)

call addition with args=(1, 2) and kwargs={}
Result: 3


3

But our decorator is very specialized here, it only works with functions taking two parameters, and will display `"Addition"` in all cases. We can modify it to make it more generic (remember `*args` and `**kwargs`).

In [8]:
def print_decorator(function):
    def new_function(*args, **kwargs):
        print(f"call {function.__name__} with args={args} and kwargs={kwargs}")
        return_value = function(*args, **kwargs)
        print(f"Result: {return_value}")
        return return_value

    return new_function

I'll let you try it out on different functions to see how generic it is.

Function definitions are not limited to one decorator: you can specify as many as you want, by placing them one after the other.



In [21]:
@my_decorator
@print_decorator
def useless():
    pass

@print_decorator
def add(a,b) : 
    return a +b

@print_decorator
def multiply(*args) : 
    result = 1
    for i in args : 
        result *= i 
    return result

add(5,4)
multiply(5,4,6,5)


new_function
call add with args=(5, 4) and kwargs={}
Result: 9
call multiply with args=(5, 4, 6, 5) and kwargs={}
Result: 600


600

The order in which they are specified is important, the previous code is equivalent to :

In [10]:
def useless():
    pass


useless = my_decorator(print_decorator(useless))

new_function


So we can see that the decorators specified first are those that will be applied last.

I said earlier that the decorators apply to functions. This is also true for functions defined within classes (i.e. methods). But you should also know that decorators extend to class declarations.

In [ ]:
@print_decorator
class MyClass:
    @my_decorator
    def method(self):
        pass

Although the definition is broader than that. The decorator is a callable taking a callable as a parameter, and can return any type of object.

We have seen how to apply a decorator to a function, however we may want to parameterize the behavior of this decorator. In our previous example (`print_decorator`), we display text before and after the function call. But what do we do if we want to change this text (to change the language, or use a term other than "function")?

We don't want to have to create a different decorator for every possible and imaginable sentence. We'd like to be able to pass our strings to our decorator so that it can display them at the right time.

In fact, `@` doesn't have to be followed by an object name, arguments can also be added to it using parentheses (as we would do for a call). But the behavior may seem strange to you at first glance.

For example, to use such a parameterized decorator :

In [22]:
@param_print_decorator("call {} with args({}) and kwargs({})", "Return value = {}")
def test_func(x):
    return x

NameError: name 'param_print_decorator' is not defined

We will need to have a callable `param_print_decorator` which, when called, will return a decorator that can then be applied to our function. **So a parametrized decorator is just a callable that returns a simple decorator.**

The `param_print_decorator` code would thus look like :

In [23]:
def param_print_decorator(string_before_execution, string_after_execution):
    def simple_decorator(function):
        def new_function(*args, **kwargs):
            print(string_before_execution.format(function.__name__, args, kwargs))
            return_value = function(*args, **kwargs)
            print(string_after_execution.format(return_value))
            return return_value

        return new_function

    return simple_decorator

A function is not just a piece of code with parameters. It also has a name, names of the parameters, documentation (docstring), annotations, etc. When we decorate a function, and where we return a new one function like in the previous example, we lose all this related information.

Here is an example to illustrate this:

In [24]:
def my_decorator(function):
    def decorated(*args, **kwargs):
        return function(*args, **kwargs)

    return decorated


@my_decorator
def addition(a, b):
    "This function makes an addition."
    return a + b


help(addition)

Help on function decorated in module __main__:

decorated(*args, **kwargs)



For more information of decorators, please read https://realpython.com/primer-on-python-decorators/